In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df=pd.read_csv('/content/phishing_site_urls.csv')

In [ ]:
df

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad
...,...,...
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [ ]:
df.isnull().sum()

,0
URL,0
Label,0


In [ ]:
!pip install tldextract tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.7 MB/s eta 0:00:00


In [ ]:
import re
import tldextract

from urllib.parse import urlparse, parse_qs
import pandas as pd
from tqdm import tqdm
from datetime import datetime


In [ ]:
SUSPICIOUS_WORDS = [
    "secure", "account", "verify",
    "login", "update", "bank", "confirm"
]


In [ ]:
def extract_features(url):
    features = {}

    parsed = urlparse(url)

    # ---- Lexical Features ----
    features["url_length"] = len(url)
    features["num_digits"] = sum(c.isdigit() for c in url)
    features["num_special_chars"] = sum(not c.isalnum() for c in url)
    features["num_dots"] = url.count(".")
    features["has_ip"] = int(bool(re.search(r"\d+\.\d+\.\d+\.\d+", url)))
    features["has_at_symbol"] = int("@" in url)
    features["has_double_slash"] = int(url.count("//") > 1)
    features["has_hyphen"] = int("-" in url)
    features["protocol_http"] = int(parsed.scheme == "http")
    features["protocol_https"] = int(parsed.scheme == "https")

    # ---- Domain & Subdomain ----
    ext = tldextract.extract(url)
    domain = ext.domain or ""
    subdomain = ext.subdomain or ""
    suffix = ext.suffix or ""

    features["domain_length"] = len(domain)
    features["subdomain_length"] = len(subdomain)
    features["num_subdomains"] = subdomain.count(".") + 1 if subdomain else 0
    features["tld_length"] = len(suffix)

    features["suspicious_subdomain"] = int(
        any(word in subdomain.lower() for word in SUSPICIOUS_WORDS)
    ) if subdomain else 0

    # ---- Path & Query ----
    features["path_length"] = len(parsed.path)
    features["query_length"] = len(parsed.query)

    params = parse_qs(parsed.query)
    features["num_params"] = len(params)

    lower_url = url.lower()
    features["has_login_keyword"] = int("login" in lower_url)
    features["has_secure_keyword"] = int("secure" in lower_url)
    features["has_update_keyword"] = int("update" in lower_url)

    # ---- Dummy WHOIS Features (placeholders) ----
    # We keep columns so your model input size doesn't change later


    return features


In [ ]:
df.keys()

Index(['URL', 'Label'], dtype='object')

In [ ]:
feature_rows = []

for u in tqdm(df["URL"], desc="Extracting Features (fast, no WHOIS)"):
    feature_rows.append(extract_features(u))

feature_df = pd.DataFrame(feature_rows)
feature_df["Label"] = df["Label"]

feature_df.head()
print("Shape:", feature_df.shape)


Extracting Features (fast, no WHOIS): 100%|██████████| 549346/549346 [00:13<00:00, 40001.17it/s]


Shape: (549346, 22)


In [ ]:
feature_df.head()

,url_length,num_digits,num_special_chars,num_dots,has_ip,has_at_symbol,has_double_slash,has_hyphen,protocol_http,protocol_https,...,num_subdomains,tld_length,suspicious_subdomain,path_length,query_length,num_params,has_login_keyword,has_secure_keyword,has_update_keyword,Label
0,225,58,32,6,0,0,0,1,0,0,...,0,2,0,134,90,4,1,0,0,bad
1,81,1,15,5,0,0,0,1,0,0,...,1,3,0,81,0,0,0,0,0,bad
2,177,47,19,7,0,0,0,1,0,0,...,0,3,0,177,0,0,0,1,1,bad
3,60,0,8,6,0,0,0,0,0,0,...,1,3,0,60,0,0,0,0,0,bad
4,116,21,13,1,0,0,0,1,0,0,...,0,3,0,79,36,0,0,0,0,bad


In [ ]:
i=feature_df['Label'].value_counts()
print(i)

Label
good    392924
bad     156422
Name: count, dtype: int64


In [ ]:
feature_df["label"] = feature_df["Label"].map({"good": 0, "bad": 1})


In [ ]:
feature_df.head()

,url_length,num_digits,num_special_chars,num_dots,has_ip,has_at_symbol,has_double_slash,has_hyphen,protocol_http,protocol_https,...,tld_length,suspicious_subdomain,path_length,query_length,num_params,has_login_keyword,has_secure_keyword,has_update_keyword,Label,label
0,225,58,32,6,0,0,0,1,0,0,...,2,0,134,90,4,1,0,0,bad,1
1,81,1,15,5,0,0,0,1,0,0,...,3,0,81,0,0,0,0,0,bad,1
2,177,47,19,7,0,0,0,1,0,0,...,3,0,177,0,0,0,1,1,bad,1
3,60,0,8,6,0,0,0,0,0,0,...,3,0,60,0,0,0,0,0,bad,1
4,116,21,13,1,0,0,0,1,0,0,...,3,0,79,36,0,0,0,0,bad,1


In [ ]:
feature_df.drop(['Label'],axis=1,inplace=True)

In [ ]:
feature_df.head()

,url_length,num_digits,num_special_chars,num_dots,has_ip,has_at_symbol,has_double_slash,has_hyphen,protocol_http,protocol_https,...,num_subdomains,tld_length,suspicious_subdomain,path_length,query_length,num_params,has_login_keyword,has_secure_keyword,has_update_keyword,label
0,225,58,32,6,0,0,0,1,0,0,...,0,2,0,134,90,4,1,0,0,1
1,81,1,15,5,0,0,0,1,0,0,...,1,3,0,81,0,0,0,0,0,1
2,177,47,19,7,0,0,0,1,0,0,...,0,3,0,177,0,0,0,1,1,1
3,60,0,8,6,0,0,0,0,0,0,...,1,3,0,60,0,0,0,0,0,1
4,116,21,13,1,0,0,0,1,0,0,...,0,3,0,79,36,0,0,0,0,1


In [ ]:
X = feature_df.drop("label", axis=1).values
y = feature_df["label"].values

np.save("X_features.npy", X)
np.save("y_labels.npy", y)


In [ ]:
print("Saved:")
print("X_features.npy →", X.shape)
print("y_labels.npy   →", y.shape)

Saved:
X_features.npy → (549346, 21)
y_labels.npy   → (549346,)


In [ ]:
# ==========================================================
# 5. CREATE FEATURE CONFIG JSON
# ==========================================================

all_features = list(feature_df.drop("label", axis=1).columns)
numeric_features = all_features.copy()   # all are numeric in your case


config = {
    "all_features": all_features,
    "numeric_features": numeric_features
}

import json
with open("feature_config.json", "w") as f:
    json.dump(config, f, indent=4)

print("Saved feature_config.json with", len(all_features), "features")


Saved feature_config.json with 21 features
